# APIs and Databases
## A very superficial intro

In this notebook we will explore how we can get data using APIs as well as where/how this data can be stored and processed.

We will look at the following technologies:

The requests library to work with REST APIs
JSON - the most commonly used format for *unstructured* data (don't confuse with unstructured as in text or images)
MongoDB - a popular NoSQL that natively handles JSON type data (we will be using MLab in the cloud rather than a local installation)

Finally we will also have a look at SQL. For better or worse SQL type databases are still around and will be around in the foreseable future. Therefore, we need to get some basics.

**For this tutorial we will need access to a MongoDB instance**

If you like to use it you can install the free [MongoDB community edition on your machine](https://docs.mongodb.com/manual/administration/install-community/)

However, it is much much easier and faster (for now) to use a hosted online version. You can get 500mb free space to play around from mLab https://mlab.com/

In [145]:
import requests as rq
import json
import time, random

In [146]:
import pandas as pd

In [149]:
from pymongo import MongoClient
# c = MongoClient() if you connect locally

# Please enter your credentials in the different fields



connection = MongoClient('ds151282.mlab.com', 51282)
db = connection['sds-fun']
db.authenticate('sds', 'aausds2018')

True

### MongoDB
MongoDB is as mentioned NoSQL which means that it uses a hierarchical structure format (for the lack of a better expression). It stores BSON (binary JSON files) as so called documents within collections within Databases.
Why is that great?

There is no schema and you can basically drop arbitrary JSON chunks into a MongoDB collection

![](https://docs.mongodb.com/manual/_images/data-model-denormalized.bakedsvg.svg)

JSON data is overall equal to Python dictionaries and thus collections of key-value pairs with nested other dictionaries and/or lists.

### Requests

The requests library allows us to interact with APIs by making GET or POST calls. Every time you post something on e.g. Facebook, your phone is making a POST requests to and Facebook API endpoint sending the text or picture along with some metadata. When obtaining data we mostly use GET requests (which is kind of logical). Actually we can use the requests GET with any kind of URL, and will receive whatever is hiding behind this URL (usually some HTML output) sent back by the server.

Note, that recurrent requests are heavy on servers and generate traffic. People runnig pages don not like that. Therefore, be nice and build in some sleep-timers into your loops when running many requests on some page. OR THE'LL BAN YOU!!!

In [150]:
response = rq.get('https://nomadlist.com/@trevorgerhardt.json')

In [151]:
response.content

b'{"success":"true","photo":"https:\\/\\/nomadlist.com\\/assets\\/img\\/users\\/139df30a327ca4af8c284e4f0ef14cd1.jpg","background":"https:\\/\\/nomadlist.com\\/assets\\/img\\/users\\/139df30a327ca4af8c284e4f0ef14cd1-bg.jpg","username":"@trevorgerhardt","location":{"now":{"city":"Canggu","country":"Indonesia","country_code":"id","latitude":-8,"longitude":115,"epoch_start":1535846400,"epoch_end":1537747200,"date_start":"2018-09-02","date_end":"2018-09-24"},"next":{"city":"Hong Kong","country":"China","country_code":"cn","latitude":22,"longitude":114,"epoch_start":1537747200,"epoch_end":1538092800,"date_start":"2018-09-24","date_end":"2018-09-28"},"previously":{"city":"Vancouver","country":"Canada","country_code":"ca","latitude":49,"longitude":-123,"epoch_start":1535673600,"epoch_end":1535846400,"date_start":"2018-08-31","date_end":"2018-09-02"}},"stats":{"cities":128,"countries":42,"followers":40,"following":21},"map":"https:\\/\\/nomadlist.com\\/assets\\/img\\/cards\\/nomadtrips-139df30

In [152]:
response_json = json.loads(response.content)

In [153]:
type(response_json)

dict

Let's bring our data into MongoDB

Most important commands for you:


```
collection = db.collection

collection.insert_one(some_dict)
collection.insert_many(sequence of dicts) # you can also pass a pandas dataframe as a list of dictionaries with .to_dict() attached

collection.count

collection.find_one()

cursor = collection.find()

```

In [164]:
# We'll create a new collection
people1 = db.people1

In [165]:
# And put in the parsed JSON
people1.insert_one(response_json)

In [167]:
# Is it in there?

people1.find_one()

Let's get some more data in and automize the "harvesting"
We can for example extract the list of all followers of our initial person
Turns out the uuids can also be used in the Nomadlist API

In [173]:
# Let's make a list of ids of people that we would like to take out of the DB
harvestlist = response_json['followers'].keys()

In [175]:
# A API friendly loop to extract the data for our 40 people

for i in harvestlist:
    q = 'https://nomadlist.com/'+str(i)+'.json' # contructs the query for the GET call
    res = rq.get(q) # grab the data form the API
    if res.status_code in [502,404]: # securety measures. Continue the loop in case an error pops up
        continue
    people1.insert_one(json.loads(res.content)) # put the data into the DB
    time.sleep(random.uniform(0.5,1)) # chill between 0.5 and 1 sec. Primitively simulate human behaviour.

KeyboardInterrupt: 

In [176]:
people1.count() #did it work?

42

In [177]:
cursor = people1.find() # Now we have the data we can take it out

In [178]:
cursor.next()

{'_id': ObjectId('5b97867a0a90b6f385ca4c62'),
 'success': 'true',
 'photo': 'https://nomadlist.com/assets/img/users/79d6a8d1d1659f6eb824844ba3e97b75.jpg',
 'background': 'https://nomadlist.com/assets/img/users/79d6a8d1d1659f6eb824844ba3e97b75-bg.jpg',
 'username': '@nefer_l',
 'location': {'now': {'city': 'Charlottesville',
   'country': 'United States',
   'country_code': 'US',
   'latitude': 38,
   'longitude': -78,
   'epoch_start': 1513296000,
   'epoch_end': 1514678400,
   'date_start': '2017-12-15',
   'date_end': '2017-12-31'},
  'next': {'city': None,
   'country': None,
   'country_code': None,
   'latitude': 0,
   'longitude': 0,
   'epoch_start': 0,
   'epoch_end': 0,
   'date_start': '1970-01-01',
   'date_end': '1970-01-01'},
  'previously': {'city': None,
   'country': None,
   'country_code': None,
   'latitude': 0,
   'longitude': 0,
   'epoch_start': 0,
   'epoch_end': 0,
   'date_start': '1970-01-01',
   'date_end': '1970-01-01'}},
 'stats': {'cities': 14, 'countries'

In [181]:
# We can be a bit more selective
cursor = people1.find({'location.now.country':'Indonesia'},{'_id':0,'username':1,'stats':1}) 

As you can see the query construction in mongo is very different from what you have seen in Python or R or what you'll find in SQL. It is all {} and not really nice. But that is to some extent due to the fact that Mongo is mostly by machines for machines. Something you'll have to learn (and/or look up) if you want to work with MongoDB.

In [182]:
# Creating a pandas DF from a Mongo cursor is however not difficult.
indonesia_df = pd.DataFrame(list(cursor))

In [183]:
indonesia_df

stats       username
0  {'cities': 15, 'countries': 9, 'followers': 1,...  @abbygmcclain
1  {'cities': 13, 'countries': 7, 'followers': 29...  @andreyazimov
2  {'cities': 62, 'countries': 34, 'followers': 1...      @tomaslau

In [184]:
# We can also unpack nested dictionaries (here the stats column)
pd.DataFrame([x[1] for x in indonesia_df.stats.iteritems()])

cities  countries  followers  following
0      15          9          1          1
1      13          7        292         49
2      62         34        152        464

Mongo has many integrated complex functions for working with "BigData". Why not inside Pandas? A database will handle data on disk rather than in memory, index things for fast access and much more. 

One really useful but unfortunately complex (I have to look it up every time I use it) is aggregation of nested elements. 

MongoDB works with so called aggregation pipelines with a killer syntax :-/ 

In the following we will try to unpack or "unwind" the trips that are nested within every user-document. Why would we do that? Because you would like to analyse travel behavior on the micro level (individual trips).
Want to know more? https://docs.mongodb.com/manual/reference/operator/aggregation-pipeline/

Below we will create one of these pipelines combining match (a filtering function), project (for selecting what should be returned) and unwind (for disaggregation of nested arrays).

In [186]:
# Return only trips of people that are in Indonesia at the moment (strange query but not wrong)
cursor = people1.aggregate([{'$match':{'location.now.country':'Indonesia'}},
                            {'$project':{'_id':0,'username':1,'trips':1}},
                            {'$unwind':'$trips'}])

In [194]:
# Or just return all trips
cursor = people1.aggregate([{'$project':{'_id':0,'username':1,'trips':1}},
                            {'$unwind':'$trips'}])

In [190]:
len(list(cursor))

1823

Unfortunately we cannot pass this directly to pandas and will have to unpack a bit using a simple loop

In [195]:
# Unpacking the returned documents. Basically we just need to enter the "trips" key. We also add the username.
trips = []
while cursor:
    doc = cursor.next()
    trip = doc['trips']
    trip['username'] = doc['username']
    trips.append(trip)

StopIteration: 

In [196]:
# Now we can create a dataframe
trips_df = pd.DataFrame(trips)

In [197]:
trips_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1823 entries, 0 to 1822
Data columns (total 17 columns):
country         1823 non-null object
country_code    1798 non-null object
country_slug    1823 non-null object
date_end        1823 non-null object
date_start      1823 non-null object
epoch_end       1823 non-null int64
epoch_start     1823 non-null int64
latitude        1823 non-null int64
length          1823 non-null object
longitude       1823 non-null int64
overlaps        1613 non-null object
place           1823 non-null object
place_photo     1823 non-null object
place_slug      1823 non-null object
place_url       1823 non-null object
user_photo      1823 non-null object
username        1823 non-null object
dtypes: int64(4), object(13)
memory usage: 242.2+ KB


In [198]:
trips_df.head()

country country_code   country_slug    date_end  date_start  \
0  United States           US  united-states  2017-12-31  2017-12-15   
1  United States           US  united-states  2017-12-15  2017-10-04   
2  United States           US  united-states  2017-10-04  2017-09-26   
3  United States           US  united-states  2017-09-25  2017-09-22   
4         Mexico           MX         mexico  2016-09-22  2016-07-12   

    epoch_end  epoch_start  latitude    length  longitude  \
0  1514678400   1513296000        38   16 days        -78   
1  1513296000   1507075200        32  2 months       -117   
2  1507075200   1506384000        43    8 days        -89   
3  1506297600   1506038400        44    3 days        -89   
4  1474502400   1468281600        20  2 months        -87   

                                            overlaps             place  \
0                                                 []   Charlottesville   
1  [{'username': '@traveladdict19', 'user_id': 'd...         San Diego   
2                                                 []           Madison   
3                                                 []          Tigerton   
4  [{'username': '@geo_care', 'user_id': 'cc6952c...  Playa del Carmen   

                                         place_photo  \
0  https://nomadlist.com/assets/img/cities/new-yo...   
1  https://nomadlist.com/assets/img/cities/san-di...   
2  https://nomadlist.com/assets/img/cities/madiso...   
3  https://nomadlist.com/assets/img/cities/new-yo...   
4  https://nomadlist.com/assets/img/cities/playa-...   

                   place_slug  \
0               united-states   
1  san-diego-ca-united-states   
2    madison-wi-united-states   
3               united-states   
4     playa-del-carmen-mexico   

                                          place_url  \
0               https://nomadlist.com/united-states   
1  https://nomadlist.com/san-diego-ca-united-states   
2    https://nomadlist.com/madison-wi-united-states   
3               https://nomadlist.com/united-states   
4     https://nomadlist.com/playa-del-carmen-mexico   

                                          user_photo  username  
0  https://nomadlist.com//assets/img/users/79d6a8...  @nefer_l  
1  https://nomadlist.com//assets/img/users/79d6a8...  @nefer_l  
2  https://nomadlist.com//assets/img/users/79d6a8...  @nefer_l  
3  https://nomadlist.com//assets/img/users/79d6a8...  @nefer_l  
4  https://nomadlist.com//assets/img/users/79d6a8...  @nefer_l

### Moving on to SQL

We will be using SQLite, a very simple SQL database (often used in mobile devices). Not as powerful as PosgreSQL or MySQL but easier to work with. 

In [204]:
# First we need to import the sqlite driver
import sqlite3

In [205]:
# Establish a connection and create a DB file on disk
db = sqlite3.connect('db_training.db')

In [206]:
# We can actually write directly from Pandas to SQL

trips_df[['country', 'country_code', 'country_slug', 'date_end', 'date_start',
       'epoch_end', 'epoch_start', 'latitude', 'length', 'longitude', 'place', 'place_photo', 'place_slug', 'place_url',
       'user_photo', 'username']].to_sql('trips', db)

In [207]:
# Let's read a bit manually
# First we find out which tables we can ses in the connected DB
cursor = db.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
cursor.fetchall()

[('trips',)]

In [210]:
# Find trips to Indonesia

mycat = db.cursor()
mycat.execute("""SELECT * FROM trips WHERE country = 'Indonesia'""")

In [209]:
len(mycat.fetchall())

72

In [211]:
# We can pass the cursor directly to Pandas (similar to MongoDB)
indonesia_df1 = pd.DataFrame(mycat.fetchall())

In [212]:
indonesia_df1

0          1   2          3           4           5           6   \
0     48  Indonesia  ID  indonesia  2016-12-31  2016-12-28  1483142400   
1     49  Indonesia  ID  indonesia  2016-12-28  2016-12-27  1482883200   
2     51  Indonesia  ID  indonesia  2016-12-21  2016-11-16  1482278400   
3     74  Indonesia  ID  indonesia  2014-12-01  2014-10-01  1417392000   
4     75  Indonesia  ID  indonesia  2014-10-01  2014-09-01  1412121600   
5    139  Indonesia  ID  indonesia  2018-02-07  2018-01-08  1517961600   
6    140  Indonesia  ID  indonesia  2018-01-08  2018-01-05  1515369600   
7    141  Indonesia  ID  indonesia  2018-01-05  2017-12-10  1515110400   
8    274  Indonesia  ID  indonesia  2016-12-05  2016-11-07  1480896000   
9    334  Indonesia  ID  indonesia  2015-05-03  2015-05-02  1430611200   
10   335  Indonesia  ID  indonesia  2015-05-03  2015-04-28  1430611200   
11   336  Indonesia  ID  indonesia  2015-05-01  2015-04-28  1430438400   
12   337  Indonesia  ID  indonesia  2015-04-24  2015-03-27  1429833600   
13   397  Indonesia  ID  indonesia  2015-10-13  2015-10-05  1444694400   
14   634  Indonesia  ID  indonesia  2016-10-23  2016-10-15  1477180800   
15   691  Indonesia  ID  indonesia  2017-03-04  2017-02-15  1488585600   
16   694  Indonesia  ID  indonesia  2017-02-03  2017-01-15  1486080000   
17   717  Indonesia  ID  indonesia  2015-10-13  2015-10-05  1444694400   
18   732  Indonesia  id  indonesia  2018-09-30  2018-09-01  1538265600   
19   761  Indonesia  ID  indonesia  2017-04-05  2017-04-03  1491350400   
20   778  Indonesia  ID  indonesia  2015-12-26  2015-12-25  1451088000   
21   779  Indonesia  ID  indonesia  2015-12-25  2015-12-24  1451001600   
22   780  Indonesia  ID  indonesia  2015-12-23  2015-12-21  1450828800   
23   829  Indonesia  ID  indonesia  2016-06-30  2016-04-16  1467244800   
24   830  Indonesia  ID  indonesia  2016-04-16  2016-03-17  1460764800   
25   834  Indonesia  ID  indonesia              2015-10-15           0   
26   937  Indonesia  ID  indonesia  2017-12-10  2017-12-01  1512864000   
27   938  Indonesia  ID  indonesia  2017-12-01  2017-10-30  1512086400   
28   992  Indonesia  id  indonesia  2018-11-05  2018-08-23  1541376000   
29   994  Indonesia  id  indonesia  2018-08-21  2018-02-27  1534809600   
..   ...        ...  ..        ...         ...         ...         ...   
42  1142  Indonesia  ID  indonesia  2017-01-06  2016-11-16  1483660800   
43  1193  Indonesia  ID  indonesia  2015-04-29  2015-04-14  1430265600   
44  1201  Indonesia  ID  indonesia  2014-12-21  2014-11-13  1419120000   
45  1202  Indonesia  ID  indonesia  2014-11-12  2014-11-12  1415750400   
46  1203  Indonesia  ID  indonesia  2014-11-12  2014-11-10  1415750400   
47  1204  Indonesia  ID  indonesia  2014-11-09  2014-11-09  1415491200   
48  1205  Indonesia  ID  indonesia  2014-11-09  2014-11-07  1415491200   
49  1216  Indonesia  ID  indonesia  2014-07-23  2014-07-20  1406073600   
50  1217  Indonesia  ID  indonesia  2014-07-20  2014-07-18  1405814400   
51  1218  Indonesia  ID  indonesia  2014-07-18  2014-07-06  1405641600   
52  1337  Indonesia  id  indonesia  2018-12-18  2018-10-18  1545091200   
53  1399  Indonesia  ID  indonesia  2018-04-18  2018-03-20  1524009600   
54  1459  Indonesia  ID  indonesia  2016-07-12  2016-07-01  1468281600   
55  1471  Indonesia  ID  indonesia  2015-07-06  2015-06-22  1436140800   
56  1494  Indonesia  id  indonesia  2019-03-05  2019-01-07  1551744000   
57  1498  Indonesia  id  indonesia  2018-11-10  2018-09-27  1541808000   
58  1501  Indonesia  ID  indonesia  2018-09-12  2018-07-16  1536710400   
59  1555  Indonesia  ID  indonesia  2017-01-28  2016-12-01  1485561600   
60  1561  Indonesia  ID  indonesia  2016-04-30  2016-04-02  1461974400   
61  1566  Indonesia  ID  indonesia  2016-01-08  2015-12-08  1452211200   
62  1567  Indonesia  ID  indonesia  2015-12-08  2015-11-10  1449532800   
63  1574  Indonesia  ID  indonesia  2015-04-17  2015-02-24  1429228800   
64  1586  

In [213]:
# We can also ask pandas to perform the query for us
indonesia_df2 = pd.read_sql_query("""SELECT * FROM trips where country = 'Indonesia'""", db)

In [215]:
indonesia_df2

index    country country_code country_slug    date_end  date_start  \
0      48  Indonesia           ID    indonesia  2016-12-31  2016-12-28   
1      49  Indonesia           ID    indonesia  2016-12-28  2016-12-27   
2      51  Indonesia           ID    indonesia  2016-12-21  2016-11-16   
3      74  Indonesia           ID    indonesia  2014-12-01  2014-10-01   
4      75  Indonesia           ID    indonesia  2014-10-01  2014-09-01   
5     139  Indonesia           ID    indonesia  2018-02-07  2018-01-08   
6     140  Indonesia           ID    indonesia  2018-01-08  2018-01-05   
7     141  Indonesia           ID    indonesia  2018-01-05  2017-12-10   
8     274  Indonesia           ID    indonesia  2016-12-05  2016-11-07   
9     334  Indonesia           ID    indonesia  2015-05-03  2015-05-02   
10    335  Indonesia           ID    indonesia  2015-05-03  2015-04-28   
11    336  Indonesia           ID    indonesia  2015-05-01  2015-04-28   
12    337  Indonesia           ID    indonesia  2015-04-24  2015-03-27   
13    397  Indonesia           ID    indonesia  2015-10-13  2015-10-05   
14    634  Indonesia           ID    indonesia  2016-10-23  2016-10-15   
15    691  Indonesia           ID    indonesia  2017-03-04  2017-02-15   
16    694  Indonesia           ID    indonesia  2017-02-03  2017-01-15   
17    717  Indonesia           ID    indonesia  2015-10-13  2015-10-05   
18    732  Indonesia           id    indonesia  2018-09-30  2018-09-01   
19    761  Indonesia           ID    indonesia  2017-04-05  2017-04-03   
20    778  Indonesia           ID    indonesia  2015-12-26  2015-12-25   
21    779  Indonesia           ID    indonesia  2015-12-25  2015-12-24   
22    780  Indonesia           ID    indonesia  2015-12-23  2015-12-21   
23    829  Indonesia           ID    indonesia  2016-06-30  2016-04-16   
24    830  Indonesia           ID    indonesia  2016-04-16  2016-03-17   
25    834  Indonesia           ID    indonesia              2015-10-15   
26    937  Indonesia           ID    indonesia  2017-12-10  2017-12-01   
27    938  Indonesia           ID    indonesia  2017-12-01  2017-10-30   
28    992  Indonesia           id    indonesia  2018-11-05  2018-08-23   
29    994  Indonesia           id    indonesia  2018-08-21  2018-02-27   
..    ...        ...          ...          ...         ...         ...   
42   1142  Indonesia           ID    indonesia  2017-01-06  2016-11-16   
43   1193  Indonesia           ID    indonesia  2015-04-29  2015-04-14   
44   1201  Indonesia           ID    indonesia  2014-12-21  2014-11-13   
45   1202  Indonesia           ID    indonesia  2014-11-12  2014-11-12   
46   1203  Indonesia           ID    indonesia  2014-11-12  2014-11-10   
47   1204  Indonesia           ID    indonesia  2014-11-09  2014-11-09   
48   1205  Indonesia           ID    indonesia  2014-11-09  2014-11-07   
49   1216  Indonesia           ID    indonesia  2014-07-23  2014-07-20   
50   1217  Indonesia           ID    indonesia  2014-07-20  2014-07-18   
51   1218  Indonesia           ID    indonesia  2014-07-18  2014-07-06   
52   1337  Indonesia           id    indonesia  2018-12-18  2018-10-18   
53   1399  Indonesia           ID    indonesia  2018-04-18  2018-03-20   
54   1459  Indonesia           ID    indonesia  2016-07-12  2016-07-01   
55   1471  Indonesia           ID    indonesia  2015-07-06  2015-06-22   
56   1494  Indonesia           id    indonesia  2019-03-05  2019-01-07   
57   1498  Indonesia           id    indonesia  2018-11-10  2018-09-27   
58   1501  Indonesia           ID    indonesia  2018-09-12  2018-07-16   
59   1555  Indonesia           ID    indonesia  2017-01-28  2016-12-01   
60   1561  Indonesia           ID    indonesia  2016-04-30  2016-04-02   
61   1566  Indonesia           ID    indonesia  2016-01-08  2015-12-08   
62   1567  Indonesia           ID    indonesia  2015-12-08  2015-11-10   
63   1574  Indonesia           ID    indonesia  2015-04-17  2015-02-24   
64   158

- If you want to learn more about working with SQL: https://www.dataquest.io/blog/python-pandas-databases/
- There is also a great intro course on Datacamp: https://www.datacamp.com/courses/intro-to-sql-for-data-science

In [216]:
# Get a cursor object
cursor = db.cursor()
cursor.execute('''
    CREATE TABLE trips_mapping('index' INTEGER PRIMARY KEY, place_slug TEXT)
''')

In [219]:
for i in indonesia_df2.iterrows():
    insert = i[1][['index','place_slug']]
    cursor.execute('''INSERT INTO trips_mapping('index', place_slug) VALUES(?,?)''', tuple(insert))

In [220]:
pd.read_sql_query("""SELECT * FROM trips_mapping""", db)

index               place_slug
0      48                indonesia
1      49    canggu-bali-indonesia
2      51    canggu-bali-indonesia
3      74      ubud-bali-indonesia
4      75                indonesia
5     139    canggu-bali-indonesia
6     140      ubud-bali-indonesia
7     141    canggu-bali-indonesia
8     274      ubud-bali-indonesia
9     334                indonesia
10    335                indonesia
11    336        jakarta-indonesia
12    337      ubud-bali-indonesia
13    397      ubud-bali-indonesia
14    634  denpasar-bali-indonesia
15    691    canggu-bali-indonesia
16    694    canggu-bali-indonesia
17    717      ubud-bali-indonesia
18    732    canggu-bali-indonesia
19    761    canggu-bali-indonesia
20    778                indonesia
21    779      ubud-bali-indonesia
22    780    canggu-bali-indonesia
23    829      ubud-bali-indonesia
24    830        bandung-indonesia
25    834      ubud-bali-indonesia
26    937    canggu-bali-indonesia
27    938      ubud-bali-indonesia
28    992    canggu-bali-indonesia
29    994    canggu-bali-indonesia
..    ...                      ...
42   1142    canggu-bali-indonesia
43   1193      ubud-bali-indonesia
44   1201      ubud-bali-indonesia
45   1202                indonesia
46   1203      ubud-bali-indonesia
47   1204                indonesia
48   1205      ubud-bali-indonesia
49   1216      ubud-bali-indonesia
50   1217                indonesia
51   1218      ubud-bali-indonesia
52   1337    canggu-bali-indonesia
53   1399    canggu-bali-indonesia
54   1459      ubud-bali-indonesia
55   1471      ubud-bali-indonesia
56   1494    canggu-bali-indonesia
57   1498    canggu-bali-indonesia
58   1501    canggu-bali-indonesia
59   1555    canggu-bali-indonesia
60   1561      ubud-bali-indonesia
61   1566      ubud-bali-indonesia
62   1567    canggu-bali-indonesia
63   1574      ubud-bali-indonesia
64   1586      ubud-bali-indonesia
65   1587                indonesia
66   1597    canggu-bali-indonesia
67   1676    canggu-bali-indonesia
68   1683    canggu-bali-indonesia
69   1696    canggu-bali-indonesia
70   1715                indonesia
71   1764        jakarta-indonesia

[72 rows x 2 columns]

In [221]:
# Close DB when finished
db.close()